In [1]:
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
from scipy.interpolate import CubicSpline
import sys
xmin = 0.01; xmax = 0.19; Npoints = 40000; step = 110
# Default kernel size is 100 microns
width = 100e-4
#!!!
def impdata(filename):
    path=f'https://raw.githubusercontent.com/homijan/ML-student-projects/intro-ab/students/bogdaale/gd-profiles/{filename}'
    #path = f'gd-profiles/{filename}'
    return path
#!!!

def getsub(f, x, xref):
    f_cs = CubicSpline(x, f, extrapolate=True)
    evalf = f_cs(xref)
    # Floor the values
    scale = 1e-4
    evalf[evalf < scale * max(evalf)] = scale * max(evalf)
    return evalf

x_Te, Te = np.loadtxt(impdata('Te_gdhohlraum_cm_10ps_TekeV_interp.txt'), usecols=(0, 1), unpack=True)
x_ne, ne = np.loadtxt(impdata('ne_gdhohlraum_cm_ne1e20cm3_interp.txt'), usecols=(0, 1), unpack=True)
x_Zbar, Zbar = np.loadtxt(impdata('Zbar_gdhohlraum_cm_Z_interp.txt'), usecols=(0, 1), unpack=True)
x_Qimpact, Qimpact = np.loadtxt(impdata('Q_gdhohlraum_microns_10ps_IMPACTWcm2.txt'), usecols=(0, 1), unpack=True)
# changing units um->cm
x_Qimpact/=1e4

# In order to include Kn
x_Qc7bBGK, Qc7bBGK, Knx = np.loadtxt(impdata('Q_gdhohlraum_cm_10ps_c7b-bgk-Wcm2-clogCHIC.txt'), comments='#', delimiter=', ', usecols=(0, 8, 6), unpack=True)
xref = np.linspace(xmin, xmax, Npoints)
T = getsub(Te, x_Te, xref)
n = getsub(ne, x_ne, xref)
Z = getsub(Zbar, x_Zbar, xref)
Qimp = getsub(Qimpact, x_Qimpact, xref)
# In order to include Kn
absKnx = -Knx
Kn = getsub(absKnx, x_Qc7bBGK, xref)

#calculating Te gradient
gradT=np.gradient(T, xref)



path = './'
data_scaling=pd.DataFrame(index=['mean', 'std'], columns=['T', 'gradT', 'Z', 'n', 'Kn', 'Q', 'beta']) #will contain mean values and std
scaled_data=pd.DataFrame([T, gradT, Z, n, Kn, Qimp], index=['T', 'gradT', 'Z', 'n', 'Kn', 'Q']).T 
# ^^^ All data of which I want to find mean and std 
for val in data_scaling.columns:
    # TODO: add beta data to scaled_data dataframe
    if (val == 'beta'):
        data_scaling[val]['mean'] = 0.0
        data_scaling[val]['std'] = 1.0
    else:
        data_scaling[val]['mean']=scaled_data[val].mean()
        data_scaling[val]['std']=scaled_data[val].std()
for col in scaled_data.columns:
    scaled_data[col]=(scaled_data[col]-data_scaling[col].loc['mean'])/data_scaling[col].loc['std']
data_scaling.to_csv(f'{path}/data_scaling.csv')
x=xref

In [2]:
T, gradT= scaled_data['T'], scaled_data['gradT'], 
Z, n= scaled_data['Z'], scaled_data['n'],
Kn, Qimp = scaled_data['Kn'], scaled_data['Q'], 
T_mean=data_scaling['T']['mean'], 
T_std=data_scaling['T']['std']

In [26]:
rad=0     #finds out how many points fits in (!)radius(!) range
s=0
while s<=width/2:
    s=xref[rad]-xref[0]
    rad+=1

numPoints = len(Z[0:2*rad:step]) #int(2*rad/step)+1 # the plus one because of integer evaluation
if (numPoints % 2 == 0):
    print(f'Number of points per field {numPoints} is not odd.')
    print('Change step or width.')
    sys.exit()
numFields = 6 # x, Z, T, gradT, Kn, n
Qdata=np.empty((0,numFields*numPoints+2), int) #2 * rad "#of points in interval" * 5 "for each phsy quantity" + 2 "for Q and beta"
print(f'Row length is {numFields*numPoints+2} (each of x, Z, T, gradT, Kn, n {numPoints} points plus Q and beta)')
print(f'Number of rows {len(x)-2*rad+1}')

x_c=np.array([]) #saves coordinate of center to index result array 

for ind, _ in enumerate(x):  #x_min=x[ind], x_max=x[ind+2*rad], x_c=x[ind+rad]
    datapoint=np.array([])          
    if ind+2*rad>=len(x)+1:
        break    

    else:
        datapoint=np.append(datapoint, T[ind:ind+2*rad:step]) #append all Te in xmin-xmax
        datapoint=np.append(datapoint, gradT[ind:ind+2*rad:step]) #append all gradTe in xmin-xmax
        datapoint=np.append(datapoint, Z[ind:ind+2*rad:step]) #append all Zbar in xmin-xmax
        datapoint=np.append(datapoint, n[ind:ind+2*rad:step]) #append all gradTe in xmin-xmax
        datapoint=np.append(datapoint, Kn[ind:ind+2*rad:step]) #append all Knudsen number in xmin-xmax
        x_c=np.append(x_c, x[rad+ind])                     #will be used as index column for Qdata
        # TODO: what is the appropriate scaling here? Global (max(x)-min(x)) might be to large!
        datapoint=np.append(datapoint, (x[ind:ind+2*rad:step] - x[rad+ind]) / (max(x) - min(x))) #append all point distances in xmin-xmax            
        datapoint=np.append(datapoint, Qimp[ind+rad]) #append Qimpact in x_c
        # Find the self-similar nonlinearity of temperature profile given by
        # T = c * (1 - b * x^2)^(1/beta),
        # where c stands for Tc, b for 1/xf^2 and beta = n in Zeldovich
        # Evaluate effective heat flux (logistic weighting of Qloc and Qfs) 
        # TODO: find how to pass x0, T0, x1, T1 as input (not global as now)
        xpoints = x[ind:ind+2*rad:step]
        Tpoints = T[ind:ind+2*rad:step].values * T_std + T_mean
        x0 = xpoints[0]; T0 = Tpoints[0]
        x1 = xpoints[len(xpoints)-1]; T1 = Tpoints[len(Tpoints)-1]
        par, cov = curve_fit(Tselfsimilar, xpoints, Tpoints, maxfev = 1000)
        # Visualize the fitting capability
        #plt.plot(xpoints, Tpoints, 'k-')
        #plt.plot(xpoints, Tselfsimilar(xpoints, par[0]), 'r--')
        #plt.show()
        #standev=np.sqrt(np.diag(cov))
        #print(f'xc {x[rad+ind]}, par {par}, Tavg {0.5*(T0 + T1)}')
        # TODO: fix this
        if (x[rad+ind]<0.15):
            par[0] = 2.5
        datapoint=np.append(datapoint, max(0.0, min(par[0], 5./2.))) #append nonlinearity of T in x_c bounded by (0,5/2)     
        # Add datapoint
        Qdata=np.append(Qdata,[datapoint], axis=0)

        if ind%500==0:
            print(f"We're done with {ind}/{len(x)-2*rad+1} points") 
                #Naming of columns 
column_names=[]
for _,name in enumerate(['T', 'gradT', 'Z', 'n', 'Kn', 'x']):
    for i in range(numPoints):
        column_names=np.append(column_names,f'{name}_{i}')
column_names=np.append(column_names,['Q', 'beta'])

df_Qdata=pd.DataFrame(Qdata, columns=column_names, index=x_c)
print('Done')

Row length is 128 (each of x, Z, T, gradT, Kn, n 21 points plus Q and beta)
Number of rows 37775
We're done with 0/37775 points
We're done with 500/37775 points
We're done with 1000/37775 points
We're done with 1500/37775 points
We're done with 2000/37775 points
We're done with 2500/37775 points


KeyboardInterrupt: 

In [ ]:

rad=0     #finds out how many points fits in (!)radius(!) range
s=0
while s<=width/2:
    s=xref[rad]-xref[0]
    rad+=1

numPoints = len(Z[0:2*rad:step]) #int(2*rad/step)+1 # the plus one because of integer evaluation
if (numPoints % 2 == 0):
    print(f'Number of points per field {numPoints} is not odd.')
    print('Change step or width.')
    sys.exit()
numFields = 6 # x, Z, T, gradT, Kn, n
Qdata=np.empty((0,numFields*numPoints+2), int) #2 * rad "#of points in interval" * 5 "for each phsy quantity" + 2 "for Q and beta"
print(f'Row length is {numFields*numPoints+2} (each of x, Z, T, gradT, Kn, n {numPoints} points plus Q and beta)')
print(f'Number of rows {len(x)-2*rad+1}')
datapoint=pd.DataFrame([T[0:0+2*rad:step],gradT[0:0+2*rad:step],Z[0:0+2*rad:step],n[0:0+2*rad:step],Kn[0:0+2*rad:step]],columns=['T','gradT', 'Z', 'n', 'Kn'])
datapoint['T']=T[0:0+2*rad:step] #append all Te in xmin-xmax
datapoint=np.append(datapoint, gradT[0:0+2*rad:step]) #append all gradTe in xmin-xmax
datapoint=np.append(datapoint, Z[0:0+2*rad:step]) #append all Zbar in xmin-xmax
datapoint=np.append(datapoint, n[0:0+2*rad:step]) #append all gradTe in xmin-xmax
datapoint=np.append(datapoint, Kn[0:0+2*rad:step]) #append all Knudsen number in xmin-xmax
x_c=np.append(x_c, x[rad+ind])                     #will be used as index column for Qdata
# TODO: what is the appropriate scaling here? Global (max(x)-min(x)) might be to large!
datapoint=np.append(datapoint, (x[ind:ind+2*rad:step] - x[rad+ind]) / (max(x) - min(x))) #append all point distances in xmin-xmax            
datapoint=np.append(datapoint, Qimp[ind+rad]) #append Qimpact in x_c
for ind, _ in enumerate(x):  #x_min=x[ind], x_max=x[ind+2*rad], x_c=x[ind+rad]

    if ind+2*rad>=len(x)+1:
        break    
##                                                      =params.values.flatten('F')
    else:
    datapoint
        # Find the self-similar nonlinearity of temperature profile given by
        # T = c * (1 - b * x^2)^(1/beta),
        # where c stands for Tc, b for 1/xf^2 and beta = n in Zeldovich
        # Evaluate effective heat flux (logistic weighting of Qloc and Qfs) 
        # TODO: find how to pass x0, T0, x1, T1 as input (not global as now)

In [ ]:

scaled_Qdata, numPoints=get_data(xref, scaled_data['T'], scaled_data['gradT'], 
                                 scaled_data['Z'], scaled_data['n'],
                                 scaled_data['Kn'], scaled_data['Q'], 
                                 width, step, 
                                 data_scaling['T']['mean'], 
                                 data_scaling['T']['std'])

In [3]:
datapoint=pd.DataFrame(columns=['T','gradT', 'Z', 'n', 'Kn'])
datapoint['T']= T[:2*rad:step]
datapoint['gradT']=gradT[:2*rad:step]
datapoint['Z']=Z[:2*rad:step]
datapoint['n']=n[:2*rad:step]
datapoint['Kn']=Kn[:2*rad:step]


NameError: name 'rad' is not defined

In [52]:
datapoint=pd.DataFrame(columns=['T','gradT', 'Z', 'n', 'Kn'])
datapoint['T']= T[:2*rad:step]
datapoint['gradT']=gradT[:2*rad:step]
datapoint['Z']=Z[:2*rad:step]
datapoint['n']=n[:2*rad:step]
datapoint['Kn']=Kn[:2*rad:step]

datapoint=datapoint.values.flatten('F')

In [53]:
datapoint

array([ 0.50184975,  0.50172719,  0.50154898,  0.50132906,  0.50107091,
        0.50076914,  0.50041367,  0.49999196,  0.4994905 ,  0.49889751,
        0.49821115,  0.49743637,  0.49657513,  0.49562991,  0.49460437,
        0.49350216,  0.49232681,  0.49108064,  0.48976489,  0.48838236,
        0.48693475,  0.32162853,  0.32153248,  0.32143082,  0.32132299,
        0.32120897,  0.32108877,  0.32096237,  0.32082957,  0.32069032,
        0.32054463,  0.32039248,  0.32023369,  0.32006762,  0.31989421,
        0.31971347,  0.3195254 ,  0.31933307,  0.31913999,  0.31894616,
        0.31875157,  0.31855642, -0.39093568, -0.39093562, -0.39093561,
       -0.39093563, -0.39093568, -0.39093574, -0.39093579, -0.39093586,
       -0.39093594, -0.390936  , -0.39093605, -0.39093611, -0.39093617,
       -0.39093622, -0.3909363 , -0.39093637, -0.39093643, -0.39093652,
       -0.39093661, -0.39093669, -0.39093678, -0.3876272 , -0.38769144,
       -0.38759004, -0.3873523 , -0.38700754, -0.38658493, -0.38

In [54]:
Kn

0       -0.408955
1       -0.408955
2       -0.408955
3       -0.408955
4       -0.408955
           ...   
39995         NaN
39996         NaN
39997         NaN
39998         NaN
39999         NaN
Name: Kn, Length: 40000, dtype: float64

In [16]:
gradT

0        0.321629
1        0.321628
2        0.321627
3        0.321626
4        0.321626
           ...   
39995    0.318031
39996    0.318046
39997    0.318061
39998    0.318076
39999    0.318084
Name: gradT, Length: 40000, dtype: float64

In [18]:
np.constants

AttributeError: module 'numpy' has no attribute 'constants'

In [20]:
from scipy import constants

In [27]:
print('{0:.1500f}'.format(constants.Boltzmann))

0.00000000000000000000001380649000000000092152160558203793320630344893869895921675384261456676071588844934012740850448608398437500000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [28]:
a=0.000000000000000000000013806490000000000921521605582037933206303448938698959216753842614566760715888449340127408504486083984375

In [29]:
a

1.380649e-23